In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

import re
import random
random.seed(2016)
pd.set_option("display.max_colwidth", 200)

In [7]:
#LOC = '/home/ec2-user/data/hd/unpacked/'
LOC = '/Users/rbekbolatov/data/kaggle/homedepot/'
df_train = pd.read_csv(LOC + 'train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(LOC + 'test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv(LOC + 'product_descriptions.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv(LOC + 'attributes.csv', encoding="ISO-8859-1")
df_matches = pd.read_csv(LOC + 'matched_strings_clean.csv').fillna("")

df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"}).fillna("EMPTY")
num_train = df_train.shape[0]
# (74067, 5), (166693, 4) -> df_train.shape, df_test.shape

In [9]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) # (240760, 5)
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')

In [81]:
df_all['brand'] #.value_counts().sum()

0                       Simpson Strong-Tie
1                       Simpson Strong-Tie
2           BEHR Premium Textured DeckOver
3                                    Delta
4                                    Delta
5                                Whirlpool
6                                Whirlpool
7                                Whirlpool
8                        Lithonia Lighting
9                            House of Fara
10                  Valley View Industries
11                                    Toro
12                                    Toro
13                                    Toro
14                             Hampton Bay
15                            InSinkErator
16                                  Sunjoy
17                    MD Building Products
18                    MD Building Products
19                    MD Building Products
20                    MD Building Products
21                           House of Fara
22                           House of Fara
23         

In [40]:
df_all[df_all['brand'].isnull()]['brand'].apply(lambda s: 1.0 if pd.isnull(s) else 0.0)

198       1.0
199       1.0
219       1.0
302       1.0
881       1.0
1208      1.0
1219      1.0
1241      1.0
1242      1.0
1419      1.0
1861      1.0
1862      1.0
1863      1.0
1864      1.0
1865      1.0
2007      1.0
2275      1.0
2276      1.0
2277      1.0
2833      1.0
3443      1.0
3563      1.0
3564      1.0
3758      1.0
4348      1.0
4578      1.0
4579      1.0
4840      1.0
5116      1.0
5513      1.0
         ... 
240167    1.0
240172    1.0
240194    1.0
240199    1.0
240206    1.0
240209    1.0
240258    1.0
240260    1.0
240276    1.0
240318    1.0
240341    1.0
240427    1.0
240428    1.0
240489    1.0
240494    1.0
240508    1.0
240511    1.0
240530    1.0
240545    1.0
240548    1.0
240560    1.0
240584    1.0
240633    1.0
240658    1.0
240670    1.0
240673    1.0
240696    1.0
240749    1.0
240757    1.0
240758    1.0
Name: brand, dtype: float64

In [55]:
def brand_features(s):
    a = [0.0, 0.0, 0.0, 0.0, 0.0]
    if pd.isnull(s):
        a[0] = 1.0
    elif s == 'Unbranded':
        a[1] = 1.0
    elif s == 'Unbranded':
        a[1] = 1.0
    elif s == 'Hampton Bay':
        a[2] = 1.0
    elif s == 'GE':
        a[3] = 1.0
    elif s == 'Everbilt':
        a[4] = 1.0
    return a

df_brand_features = pd.DataFrame(df_all['brand'].apply(lambda s: brand_features(s)).tolist())

df_train = df_brand_features.iloc[:num_train]
df_test = df_brand_features.iloc[num_train:]

df_train

df_train.head()
#df_all['brand'].apply(lambda s: brand_features(s))

,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0


In [66]:
df_train = np.array(df_train)
df_test = np.array(df_test)

In [68]:
np.save('features_brand_01_train', df_train)
np.save('features_brand_01_test', df_test)

In [60]:
df_train.to_csv('features_brand_01_train.csv')
df_test.to_csv('features_brand_01_test.csv')

In [46]:
df_train.head()

0    [0.0, 0.0, 0.0, 0.0, 0.0]
1    [0.0, 0.0, 0.0, 0.0, 0.0]
2    [0.0, 0.0, 0.0, 0.0, 0.0]
3    [0.0, 0.0, 0.0, 0.0, 0.0]
4    [0.0, 0.0, 0.0, 0.0, 0.0]
Name: brand, dtype: object

In [59]:
a.value_counts()

Unbranded                     7766
Hampton Bay                   5173
GE                            3012
Everbilt                      2819
Home Decorators Collection    2608
KOHLER                        2580
DEWALT                        2034
Delta                         2029
Ryobi                         1903
Prime-Line                    1754
Philips                       1634
Glacier Bay                   1631
Milwaukee                     1550
Veranda                       1387
RIDGID                        1359
Whirlpool                     1316
Crown Bolt                    1294
Husky                         1247
MOEN                          1243
American Standard             1240
Daltile                       1212
Lithonia Lighting             1178
LG Electronics                1110
HDX                           1077
MS International               971
Leviton                        913
Makita                         877
BLACK+DECKER                   872
Masonite            

In [4]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) # (240760, 5)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
df_all = pd.merge(df_all, df_matches, on='id')

In [5]:
df_attr_len = df_attr.groupby('product_uid', as_index=False)['name'].agg({'attr_count':(lambda x: len(list(x)))})
df_all = pd.merge(df_all, df_attr_len, how='left', on='product_uid')
df_all['attr_count'] = df_all['attr_count'].fillna(0)

In [8]:
#df_all.drop('list', axis=1, inplace=True)
df_all.columns

Index([                 u'id',       u'product_title',         u'product_uid',
                 u'relevance',         u'search_term', u'product_description',
                     u'brand',                 u'tit',                u'tit2',
                      u'desc',               u'desc2',          u'attributes',
                  u'mfgbrand',           u'mfgbrand2',          u'attr_count'],
      dtype='object')

In [9]:
pattern_camel = re.compile(r"([a-z]+)([0-9A]|([A-Z][^ ]+))")
pattern_lcase_number = re.compile(r"([a-z])([0-9])")
pattern_digit_lcase = re.compile(r"([0-9])([a-z])")
pattern_s = re.compile(r"([a-z])'s")
pattern_number_commas = re.compile(r"([0-9]),([0-9])")

    
# 4x2
XBY = "xby"
pattern_xby_d = re.compile(r"(x[0-9])")
pattern_d_xby = re.compile(r"([0-9])x")

# units
pattern_inch = re.compile(r"([0-9])( *)(inches|inch|in|')\.?")
pattern_foot = re.compile(r"([0-9])( *)(foot|feet|ft|''|\")\.?")
pattern_pound = re.compile(r"([0-9])( *)(pounds|pound|lbs|lb)\.?")
pattern_sqft = re.compile(r"([0-9])( *)(square|sq) ?\.?(feet|foot|ft)\.?")
pattern_gallons = re.compile(r"([0-9])( *)(gallons|gallon|gal)\.?")
pattern_oz = re.compile(r"([0-9])( *)(ounces|ounce|oz)\.?")
pattern_cm = re.compile(r"([0-9])( *)(centimeters|cm)\.?")
pattern_mm = re.compile(r"([0-9])( *)(milimeters|mm)\.?")
pattern_deg = re.compile(r"([0-9])( *)(degrees|degree)\.?")
pattern_volt = re.compile(r"([0-9])( *)(volts|volt)\.?")
pattern_watt = re.compile(r"([0-9])( *)(watts|watt)\.?")
pattern_amp = re.compile(r"([0-9])( *)(amperes|ampere|amps|amp)\.?")
pattern_kamp = re.compile(r"([0-9])( *)(kiloamperes|kiloampere|kamps|kamp|ka)\.?")

# split
pattern_split = re.compile('[^0-9a-z]')

known_words = set(["the", "a", "an",
    "this", "that", "which", "whose",
    "other", "and", "or",
    "be", "is", "are", "been",
    "have", "has", "had",
    "can", "could", "will", "would",
    "go", "gone", "see", "seen",
    "all", "some", "any", "most", "several", "no", "none", "nothing",
    "as", "of", "in", "on", "at", "over", "from", "to",
    "with", "through", "for", "when", "then",
    "new", "old",
    "you", "your", "yours", "me", "i", "my", "mine", "it", "its"])

def str_stem(s): 
    if isinstance(s, str) or isinstance(s, unicode):
        
        s = pattern_camel.sub(r"\1 \2", s)
        s = pattern_lcase_number.sub(r"\1 \2", s)
        s = pattern_digit_lcase.sub(r"\1 \2", s)
        s = pattern_number_commas.sub(r"\1\2", s)
        s = pattern_s.sub(r"\1", s)
        
        
        s = s.lower().strip()
        
        # 4ft x 2ft
        s = s.replace(" x "," " + XBY + " ")
        s = s.replace("*"," " + XBY + " ")        
        s = s.replace(" by "," " + XBY)
        s = pattern_xby_d.sub(" " + XBY + " \1", s)
        s = pattern_d_xby.sub("\1 " + XBY + " ", s)
        
        # units
        s = pattern_inch.sub(r"\1 inch ", s)
        s = pattern_foot.sub(r"\1 foot ", s)
        s = pattern_pound.sub(r"\1 pound ", s)
        s = pattern_sqft.sub(r"\1 sqft ", s)
        s = pattern_gallons.sub(r"\1 gal ", s)
        s = pattern_oz.sub(r"\1 oz ", s)
        s = pattern_cm.sub(r"\1 cm ", s)
        s = pattern_mm.sub(r"\1 mm ", s)
        s = pattern_deg.sub(r"\1 deg ", s)
        s = pattern_volt.sub(r"\1 volt ", s)
        s = pattern_watt.sub(r"\1 watt ", s)
        s = pattern_amp.sub(r"\1 amp ", s)
        s = pattern_kamp.sub(r"\1 kamp ", s)
        
        # some by hand
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        s = s.replace("pressure-treated","pressure treated pt")
        
        s = ' '.join([x for x in pattern_split.split(s) if x and x not in known_words])
        return s
    else:
        #raise ValueError("Type of " + str(s) + " is " + str(type(s)))
        #print "HUY"
        return 'null'
    
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))

In [10]:
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [11]:
# id	product_title	product_uid	relevance	search_term	product_description	brand
# id, relevance, search_term, product_title, product_description, (product_uid,) brand  [product_info, attr]
class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['id','relevance','search_term','product_title','product_description','product_info','attr','brand'] + \
        ['tit', 'tit2', 'desc', 'desc2', 'attributes', 'mfgbrand', 'mfgbrand2'] + \
        ['brand_feature'] #['ratio_brand']
        #[] #['ratio_title', 'ratio_description', 'ratio_brand']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

def fmse(ground_truth, predictions):
    return mean_squared_error(ground_truth, predictions)

#RMSE  = make_scorer(fmse, greater_is_better=False)
RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

In [12]:
def last_word(r):
    title = r['product_title'].split()
    ms = r['tit'].split(",")
    ic = -1
    for m in ms:
        for i, t in enumerate(title):
            if m == t and i > ic:
                ic = i
    if ic < 0:
        ic = 10
    else:
        ic = len(title) - ic - 1
    return ic #, ms, title

In [13]:
def last_word_match(r):
    title = r['product_title'].split()
    ms = r['tit'].split(",")
    if len(title) > 0 and len(ms) > 0 and title[-1] == ms[-1]:
        return 1.0
    else:
        return 0.0

In [14]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word, NWORDS):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words, NWORDS): return set(w for w in words if w in NWORDS)

def correct(word, NWORDS):
    candidates = known([word], NWORDS) or known(edits1(word), NWORDS) or known_edits2(word, NWORDS) or [word]
    return max(candidates, key=NWORDS.get)

In [15]:
already_matched = {}
def similar_words(w1, w2):
    if (w1, w2) in already_matched:
        return already_matched[(w1, w2)]
    if abs(len(w1) - len(w2)) > 2:
        res = False
    elif len(set(w1).difference(set(w2))) > 2 or len(set(w2).difference(set(w1))) > 2 :
        res = False
    else:
        res = w2 == correct(w1, {w2:1})
    already_matched[(w1, w2)] = res
    return res
    
def last_word_match_query_last(r):
    title = r['product_title'].split(' ')
    ms = r['search_term'].split(' ')
    if len(title) > 0 and len(ms) > 0 and similar_words(title[-1], ms[-1]):
        return 1.0
    else:
        return 0.0

In [16]:
start_time = time.time()

#comment out the lines below use df_all.csv for further grid search testing
#if adding features consider any drops on the 'cust_regression_vals' class

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']

df_all['len_of_query'] = df_all['search_term'].map(lambda x: max(1, len(x.split()))).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x: len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)

df_all['letters_query'] = df_all['search_term'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_title'] = df_all['product_title'].map(lambda x:len(x)).astype(np.int64)
df_all['letters_desc'] = df_all['product_description'].map(lambda x:len(x)).astype(np.int64)
df_all['letters_brand'] = df_all['brand'].map(lambda x:len(x)).astype(np.int64)

###############################
# df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
# df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))

# df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
# df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
# df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))

df_all['query_in_title'] = df_all['tit'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_description'] = df_all['desc'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_attrs'] = df_all['attributes'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_brand'] = df_all['mfgbrand'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))

df_all['letters_query_in_title'] = df_all['tit'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_description'] = df_all['desc'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_attrs'] = df_all['attributes'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_brand'] = df_all['mfgbrand'].map(lambda x: len(x)).astype(np.int64)


df_all['query_in_title2'] = df_all['tit2'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_description2'] = df_all['desc2'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_brand2'] = df_all['mfgbrand2'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))


df_all['letters_query_in_title2'] = df_all['tit2'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_description2'] = df_all['desc2'].map(lambda x: len(x)).astype(np.int64)

df_all['ratio_letters_query_in_title'] = df_all['letters_query_in_title2']/(df_all['letters_query_in_title'] + 1)
df_all['ratio_letters_query_in_descr'] = df_all['letters_query_in_description2']/(df_all['letters_query_in_description'] + 1)


df_all['query_in_title_num'] = df_all['tit'].map(lambda y: len([x for x in y.split(",") if x and re.match(r'^[0-9]+$', x)]))
df_all['query_in_description_num'] = df_all['desc'].map(lambda y: len([x for x in y.split(",") if x and re.match(r'^[0-9]+$', x)]))
df_all['query_in_attrs_num'] = df_all['attributes'].map(lambda y: len([x for x in y.split(",") if x and re.match(r'^[0-9]+$', x)]))



#df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
#df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
#df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
###############################


df_all['ratio_title'] = df_all['query_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['query_in_description']/df_all['len_of_query']

# df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
# df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
# df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']



df_all['title_match_last_title_word'] = df_all.apply(last_word, axis=1)
df_all['title_match_last_word'] = df_all.apply(last_word_match, axis=1)
df_all['title_match_last_word_query'] = df_all.apply(last_word_match_query_last, axis=1)

df_brand = pd.unique(df_all.brand.ravel())
d={}
i = 1
for s in df_brand:
    d[s]=i
    i+=1
df_all['brand_feature'] = df_all['brand'].map(lambda x: d[x])
#df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))

#df_all.to_csv('df_all_322_1.csv')
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)

df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train = df_train[:]
X_test = df_test[:]
print("--- Features Set: %s minutes ---" % round(((time.time() - start_time)/60), 2))

--- Features Set: 3.31 minutes ---


In [ ]:
df_all[1000:1030]
#df_all['query_in_title'] + 1
#df_all['query_in_title2']/(df_all['query_in_title'] + 1)

In [84]:
# LOAD FROM SAVED
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)
#df_all = pd.read_csv('df_all_322_1.csv', encoding="ISO-8859-1", index_col=0)

df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train = df_train[:]
X_test = df_test[:]

In [85]:
X_train

,id,product_title,product_uid,relevance,search_term,brand
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,Simpson Strong-Tie
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,Simpson Strong-Tie
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 Tugboat Wood and Concrete Coating,100002,3.00,deck over,BEHR Premium Textured DeckOver
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome (Valve Not Included),100005,2.33,rain shower head,Delta
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome (Valve Not Included),100005,2.67,shower only faucet,Delta
5,18,Whirlpool 1.9 cu. ft. Over the Range Convection Microwave in Stainless Steel with Sensor Cooking,100006,3.00,convection otr,Whirlpool
6,20,Whirlpool 1.9 cu. ft. Over the Range Convection Microwave in Stainless Steel with Sensor Cooking,100006,2.67,microwave over stove,Whirlpool
7,21,Whirlpool 1.9 cu. ft. Over the Range Convection Microwave in Stainless Steel with Sensor Cooking,100006,3.00,microwaves,Whirlpool
8,23,Lithonia Lighting Quantum 2-Light Black LED Emergency Fixture Unit,100007,2.67,emergency light,Lithonia Lighting
9,27,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Fluted Casing,100009,3.00,mdf 3/4,House of Fara


In [ ]:
#df_all[300:320][['relevance', 'product_title', 'search_term', 'product_description']]
#X_train.columns

In [32]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
tsvd = TruncatedSVD(n_components=10, random_state = 2016)
# from sklearn.feature_extraction import DictVectorizer
# dictvect = DictVectorizer()
from sklearn.preprocessing import OneHotEncoder
ohenc = OneHotEncoder()
randomForestRegressor = RandomForestRegressor(n_estimators = 500, min_samples_leaf=3, n_jobs = -1, random_state = 5017, verbose = 1)

clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                    
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                    
#                         ('txt1', pipeline.Pipeline([ ('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf)  ])),
#                         ('txt2', pipeline.Pipeline([ ('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf)  ])),
#                         ('txt3', pipeline.Pipeline([ ('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf) ])),
#                         ('txt4', pipeline.Pipeline([ ('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf) ]))
                    
#                         ('brandf', pipeline.Pipeline([ ('s5', cust_txt_col(key='brand_feature')), ('ohenc', ohenc)  ])),
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        'txt3': 0.5,
                        'txt4': 0.5
#                         'brandf': 1.0
                        },
                n_jobs = -1
                ))
#         , 
#         ('rfr', randomForestRegressor)
    ])

#clf.set_params(rfr__max_features=10, rfr__max_depth=20)
#clf.fit(X_train, y_train)
# X_train

In [34]:
a = clf.fit_transform(X_train)
#model.fit(X_train, y_train)
a

array([[  1.00001000e+05,   1.50000000e+01,   2.00000000e+00, ...,
          6.49899457e-05,  -1.09856277e-03,  -6.84634068e-03],
       [  1.00001000e+05,   1.50000000e+01,   2.00000000e+00, ...,
         -3.87713348e-06,  -1.13283073e-03,  -6.72812595e-03],
       [  1.00002000e+05,   3.50000000e+01,   1.00000000e+00, ...,
         -5.63536932e-03,   2.92041213e-03,  -5.97269368e-03],
       ..., 
       [  2.06641000e+05,   4.80000000e+01,   6.00000000e+00, ...,
         -1.36185457e-06,   2.63435202e-05,   8.18920971e-06],
       [  2.06648000e+05,   1.40000000e+01,   3.00000000e+00, ...,
         -6.40421081e-16,  -3.94993107e-16,  -1.12821783e-15],
       [  2.06650000e+05,   3.30000000e+01,   5.00000000e+00, ...,
         -7.15176477e-10,  -7.51147525e-08,   5.51772459e-08]])

In [38]:
b = clf.transform(X_test)

In [35]:
import xgboost as xgb

In [36]:
dtrain = xgb.DMatrix( a, label=y_train)

In [37]:
dtrain.save_binary("train.buffer")

In [39]:
dtest = xgb.DMatrix(b)
dtest.save_binary("test.buffer")

In [44]:
evallist  = [(dtrain,'train')]

In [96]:
param = {'max_depth':2, 
         'eta':3, 
#          'objective':'reg:linear',
         'eval_metric':'rmse',
         'maximize': False,
         'colsample_bytree':1,
         'subsample':1,
         'nthread':4
        }

In [77]:
num_round = 4
bst = xgb.train( param, dtrain, num_round, evallist, early_stopping_rounds=15, verbose_eval=2)
bst.best_iteration, bst.best_ntree_limit

(3, 4)

In [97]:
#plst = list(param.items())
xgb.cv(param, dtrain, num_boost_round=num_round, nfold=2, metrics={'error'}, seed = 0)

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,-1.381635,0.002793,-1.381635,0.002793
1,-1.381635,0.002793,-1.381635,0.002793
2,-1.381635,0.002793,-1.381635,0.002793
3,-1.381635,0.002793,-1.381635,0.002793


In [105]:
clf = xgb.XGBClassifier(param)
paramso = {'max_depth': [2], 
         'learning_rate': [3], 
         'colsample_bytree':[1],
         'subsample':[1]
        }
clf.get_params().keys()

['reg_alpha',
 'colsample_bytree',
 'silent',
 'colsample_bylevel',
 'scale_pos_weight',
 'learning_rate',
 'missing',
 'max_delta_step',
 'nthread',
 'base_score',
 'n_estimators',
 'subsample',
 'reg_lambda',
 'seed',
 'min_child_weight',
 'objective',
 'max_depth',
 'gamma']

In [ ]:
model = grid_search.GridSearchCV(estimator = clf, param_grid = paramso, n_jobs = -1, cv = 5, verbose = 20, scoring=RMSE)
model.fit(a, y_train)

print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [55]:
predicted = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [56]:
predicted

array([ 2.27597213,  2.03546667,  2.45794249, ...,  1.62093401,
        2.60387492,  2.5656836 ], dtype=float32)

In [95]:
param_grid = {'max_features': [2], 'max_depth': [30]}
model = grid_search.GridSearchCV(estimator = randomForestRegressor, param_grid = param_grid, n_jobs = -1, cv = 5, verbose = 20, scoring=RMSE)
model.fit(a, y_train)

print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  1.5min remaining:  -15.2s
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  1.5min remaining:  -15.2s
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  1.5min remaining:  -15.2s
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  1.9min remaining:  -19.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | el

Best parameters found by grid search:
{'max_features': 2, 'max_depth': 30}
Best CV score:
-0.466961553969
[CV] max_features=2, max_depth=30 ....................................
[CV] max_features=2, max_depth=30 ....................................
[CV] max_features=2, max_depth=30 ....................................
[CV] max_features=2, max_depth=30 ....................................
[CV] .......... max_features=2, max_depth=30, score=-0.474227 - 1.5min[CV] .......... max_features=2, max_depth=30, score=-0.461016 - 1.5min[CV] .......... max_features=2, max_depth=30, score=-0.455927 - 1.5min[CV] .......... max_features=2, max_depth=30, score=-0.464262 - 1.5min



[CV] max_features=2, max_depth=30 ....................................
[CV] .......... max_features=2, max_depth=30, score=-0.479375 -  22.6s


In [30]:
start_time = time.time()

param_grid = {'rfr__max_features': [2], 'rfr__max_depth': [30]}
model = grid_search.GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 5, verbose = 20, scoring=RMSE)
model.fit(X_train, y_train)

print("--- Training: %s minutes ---" % round(((time.time() - start_time)/60),2))

print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  2.3min remaining:  -23.3s
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  2.3min remaining:  -23.5s
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  2.4min remaining:  -23.8s
[Parallel(n_jobs=-1)]: Done   6 out of   5 | elapsed:  3.1min remaining:  -30.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.1min finished
/usr/local/lib64/python2.7/site-packages/sklearn/pipeline.py:497: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for name, trans in self.transformer_list)
/usr/local/lib64/python2.7/site-packages/sklearn/pipeline.py:497: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for name, trans in self.transformer_list)
/usr/local/lib64/python2.7/site-packages/sklearn/pipeline.py:497: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1

[CV] rfr__max_features=2, rfr__max_depth=30 ..........................
[CV] rfr__max_features=2, rfr__max_depth=30 ..........................
[CV] rfr__max_features=2, rfr__max_depth=30 ..........................
[CV] rfr__max_features=2, rfr__max_depth=30 ..........................
[CV]  rfr__max_features=2, rfr__max_depth=30, score=-0.481643 - 2.2min[CV]  rfr__max_features=2, rfr__max_depth=30, score=-0.466586 - 2.3min[CV]  rfr__max_features=2, rfr__max_depth=30, score=-0.461499 - 2.2min[CV]  rfr__max_features=2, rfr__max_depth=30, score=-0.469350 - 2.2min



[CV] rfr__max_features=2, rfr__max_depth=30 ..........................
[CV]  rfr__max_features=2, rfr__max_depth=30, score=-0.486730 -  45.7s


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   31.5s finished


--- Training: 4.04 minutes ---
Best parameters found by grid search:
{'rfr__max_features': 2, 'rfr__max_depth': 30}
Best CV score:
-0.473161739747


In [ ]:
#X_train.ix[3782]
inds = pd.isnull(X_train).any(1).nonzero()[0]
inds

In [ ]:
X_train[0:3]

In [ ]:
np.isfinite(X_train.sum())

In [ ]:
np.isfinite(X_train).all()

In [ ]:
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

In [20]:
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

y_pred = model.predict(X_test)
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_x.csv',index=False)

Best parameters found by grid search:
{'rfr__max_features': 2, 'rfr__max_depth': 30}
Best CV score:
-0.468286123481


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    6.0s finished


In [ ]:
# now blending

In [23]:
fmi = pd.read_csv('./submission6_50t_50m.csv', encoding="ISO-8859-1")
fo = pd.read_csv('./submission_x.csv', encoding="ISO-8859-1")

In [24]:
fmi.head()

,id,relevance
0,1,2.006440
1,4,2.045906
2,5,2.149894
3,6,2.496521
4,7,2.222372


In [25]:
fo.head()

,id,relevance
0,1,2.055402
1,4,2.163738
2,5,2.175020
3,6,2.651792
4,7,2.345252


In [26]:
res = pd.merge(fo, fmi, how='left', on='id')

In [27]:
res.head()

,id,relevance_x,relevance_y
0,1,2.055402,2.006440
1,4,2.163738,2.045906
2,5,2.175020,2.149894
3,6,2.651792,2.496521
4,7,2.345252,2.222372


In [28]:
def f(r):
    return r['relevance_x']*0.1 + r['relevance_y']*0.9

y_pred = res.apply(f, axis=1)
pd.DataFrame({"id": res['id'], "relevance": y_pred}).to_csv('submission_x_10_90.csv',index=False)


In [6]:
df_brand

,product_uid,brand
9,100001.0,Simpson Strong-Tie
37,100002.0,BEHR Premium Textured DeckOver
69,100003.0,STERLING
93,100004.0,Grape Solar
122,100005.0,Delta
163,100006.0,Whirlpool
204,100007.0,Lithonia Lighting
236,100008.0,Teks
256,100009.0,House of Fara
283,100010.0,Valley View Industries


In [70]:
fmi = pd.read_csv('./submission10_60G_40XG.csv.gz', encoding="ISO-8859-1")

In [71]:
fmi.head()

,id,relevance
0,1,2.048145
1,4,2.071236
2,5,2.223909
3,6,2.612672
4,7,2.274974


In [72]:
def froi(x):
    if x > 2.85:
        return 3.0
    elif x > 2.55:
        return 2.66
    elif x < 1.15:
        return 1.0
    elif x < 1.45:
        return 1.33
    elif x < 2.15 and x > 1.85:
        return 2.0
    else:
        return x

In [73]:
fmi['relevance'] = fmi['relevance'].apply(froi)

In [74]:
fmi.head()

,id,relevance
0,1,2.000000
1,4,2.000000
2,5,2.223909
3,6,2.660000
4,7,2.274974


In [75]:
fmi.to_csv('submission_collapse_some_1.csv',index=False)

In [76]:
import google_spell
from google_spell import spell_check_dict

In [79]:
'congolium' in spell_check_dict

True

In [80]:
spell_check_dict['congolium']

'congoleum'